**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/select-from-where).**

---


# Introduction

Try writing some **SELECT** statements of your own to explore a large dataset of air pollution measurements.

Run the cell below to set up the feedback system.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The code cell below fetches the `global_air_quality` table from the `openaq` dataset.  We also preview the first five rows of the table.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Płock, ul. Reja",Płock,PL,bc,0.30000,2022-05-09 18:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
1,"Legnica, al. Rzeczypospolitej",Legnica,PL,bc,0.67909,2022-05-16 05:00:00+00:00,µg/m³,GIOS,1.0,51.204503,16.180513,POINT(51.204503 1)
2,"Włocławek, ul. Okrzei",Włocławek,PL,bc,2.64000,2022-04-29 06:00:00+00:00,µg/m³,GIOS,1.0,52.658467,19.059314,POINT(52.658467 1)
3,"Zielonka, Bory Tucholskie",Zielonka,PL,bc,0.41000,2022-05-04 08:00:00+00:00,µg/m³,GIOS,1.0,53.662136,17.933986,POINT(53.662136 1)
4,"Kędzierzyn-Koźle, ul. Śmiałego",Kędzierzyn-Koźle,PL,bc,1.32639,2022-05-14 21:00:00+00:00,µg/m³,GIOS,1.0,50.349608,18.236575,POINT(50.349608 1)


In [3]:
# Query to get the score column from every row where the type column has value "job"
query = """
        SELECT country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE value = 0 
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 67135368 bytes.


# Exercises

### 1) Units of measurement

Which countries have reported pollution levels in units of "ppm"?  In the code cell below, set `first_query` to an SQL query that pulls the appropriate entries from the `country` column.

In case it's useful to see an example query, here's some code from the tutorial:

```
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
```

In [4]:
# Query to select countries with units of "ppm"
first_query = """
        SELECT DISTINCT country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE unit = "ppm" 
        """  #Your code goes here

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()

# View top few rows of results
print(first_results.head())

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


  country
0      AR
1      IL
2      TW
3      CO
4      PE


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [5]:
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

first_query = """
              SELECT country
              FROM `bigquery-public-data.openaq.global_air_quality`
              WHERE unit = "ppm"
              """

# Or to get each country just once, you could use
first_query = """
              SELECT DISTINCT country
              FROM `bigquery-public-data.openaq.global_air_quality`
              WHERE unit = "ppm"
              """

```

### 2) High air quality

Which pollution levels were reported to be exactly 0?  
- Set `zero_pollution_query` to select **all columns** of the rows where the `value` column is 0.
- Set `zero_pollution_results` to a pandas DataFrame containing the query results.

In [6]:
# Query to select all columns where pollution levels are exactly 0
zero_pollution_query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE value = 0 
        """ #Your code goes here

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

# API request - run the query and return a pandas DataFrame
zero_pollution_results = query_job.to_dataframe() # Your code goes here

print(zero_pollution_results.head())

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


                                location      city country pollutant  value  \
0  Wrocław, wyb. Conrada-Korzeniowskiego   Wrocław      PL        bc    0.0   
1           Koszalin, ul. Armii Krajowej  Koszalin      PL        bc    0.0   
2           Koszalin, ul. Armii Krajowej  Koszalin      PL        bc    0.0   
3           Koszalin, ul. Armii Krajowej  Koszalin      PL        bc    0.0   
4             Żary, ul. Szymanowskiego 8      Żary      PL        bc    0.0   

                  timestamp   unit source_name  latitude  longitude  \
0 2022-04-30 06:00:00+00:00  µg/m³        GIOS       1.0  51.129378   
1 2022-05-05 01:00:00+00:00  µg/m³        GIOS       1.0  54.193986   
2 2022-05-12 02:00:00+00:00  µg/m³        GIOS       1.0  54.193986   
3 2022-05-15 17:00:00+00:00  µg/m³        GIOS       1.0  54.193986   
4 2022-05-06 15:00:00+00:00  µg/m³        GIOS       1.0  51.642656   

   averaged_over_in_hours       location_geom  
0               17.029250  POINT(51.129378 1)  
1 

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [7]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

zero_pollution_query = """
                       SELECT *
                       FROM `bigquery-public-data.openaq.global_air_quality`
                       WHERE value = 0
                       """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

zero_pollution_results = query_job.to_dataframe()

```

That query wasn't too complicated, and it got the data you want. But these **SELECT** queries don't organizing data in a way that answers the most interesting questions. For that, we'll need the **GROUP BY** command. 

If you know how to use [`groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) in pandas, this is similar. But BigQuery works quickly with far larger datasets.

Fortunately, that's next.

# Keep going
**[GROUP BY](https://www.kaggle.com/dansbecker/group-by-having-count)** clauses and their extensions give you the power to pull interesting statistics out of data, rather than receiving it in just its raw format.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-sql/discussion) to chat with other learners.*